In [20]:
import os
os.chdir("..")

In [21]:
# Import libraries
import os
import logging
from enum import Enum

import operator
from pydantic import BaseModel, Field
from typing import Annotated, Dict, List, Sequence, Literal, Optional

from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_tavily import TavilySearch
from langchain_core.prompts import PromptTemplate
from langgraph.graph import StateGraph, START, END, add_messages
from langchain_core.messages import ToolMessage, SystemMessage, BaseMessage, HumanMessage

from src.config.settings import settings
from src.agent.utils import get_date_string

In [22]:
# Set langsmith project
os.environ["LANGSMITH_API_KEY"] = settings.langsmith_api_key.get_secret_value()
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = settings.app_name

# Suppress all debug logs from urllib3 and langsmith
logging.getLogger("urllib3").setLevel(logging.WARNING)
logging.getLogger("langsmith").setLevel(logging.WARNING)
logging.getLogger("openai._base_client").setLevel(logging.WARNING)
logging.getLogger("openai._base_client").setLevel(logging.WARNING)
logging.getLogger("httpcore").setLevel(logging.WARNING)
logging.getLogger("httpx").setLevel(logging.WARNING)
logging.getLogger("websockets.client").setLevel(logging.WARNING)
logging.getLogger("pyppeteer").setLevel(logging.WARNING)

---
## 1. State

In [23]:
# Campaign schema
class CampaignGoal(str, Enum):
    """Possible objectives for an advertising campaign"""
    AWARENESS = 'awareness'
    TRAFFIC = 'traffic'
    ENGAGEMENT = 'engagement'
    LEADS = 'leads'
    APP_INSTALLS = 'app installs'


class AdPlatform(str, Enum):
    """Specific ad placements across different social platforms"""
    INSTAGRAM_REELS = "instagram reels"
    INSTAGRAM_STORY = "instagram video story"
    FACEBOOK_FEEDS = "facebook feed"
    FACEBOOK_STORY = "facebook story"
    YOUTUBE_SHORT = "youtube short"
    TIKTOK_REELS = "tiktok reels"


class Campaign(BaseModel):
    """Schema for campaign information"""
    goal: CampaignGoal = Field(
        ...,
        description="Objective for the marketing campaign"
    )
    platform: AdPlatform = Field(
        ...,
        description="Specific ad placement for the marketing campaign"
    )

In [24]:
# Target audience schema
class Gender(str, Enum):
    """
    Gender identity options for the target audience.
    """
    MALE = "male"
    FEMALE = "female"
    ALL = "male and female"


class Location(str, Enum):
    """
    Location options for the target audience.
    """
    USA = "united states"
    UK = "united kingdom"
    CANADA = "canada"
    AUSTRALIA = "australia"


class IncomeRange(str, Enum):
    """
    Income range categories for audience segmentation.
    """
    LOW = "below $30k"
    LOWER_MIDDLE = "$30k to $60k"
    MIDDLE = "$60k to $100k"
    UPPER_MIDDLE = "$100k to $200k"
    HIGH = "above $200k"


class AgeRange(BaseModel):
    min_age: int = Field(
        ...,
        description="Minimum age for the target audience"
    )
    max_age: int = Field(
        ...,
        description="Maximum age for the target audience"
    )


class TargetAudience(BaseModel):
    """Audience information schema"""
    gender: Gender = Field(
        ...,
        description="Gender identity for the target audience"
    )
    location: Location = Field(
        ...,
        description="Location for the target audience"
    )
    income_range: IncomeRange = Field(
        ...,
        description="Income range for the target audience"
    )
    age_range: AgeRange = Field(
        ...,
        description="Age range for the target audience"
    )

In [25]:
# Product schema
class ProductPlatform(str, Enum):
    """Mobile platforms supported by the product."""
    ios = "ios"
    android = "android"
    web = "web"


class Product(BaseModel):
    """Product information"""
    name: str = Field(
        ...,
        description="Name of the product"
    )
    description: str = Field(
        ...,
        description="A brief overview of the product"
    )
    features: Dict[str, str] = Field(
        ...,
        description="Dictionary mapping feature names to their description"
    )
    supported_platforms: List[ProductPlatform] = Field(
        ...,
        description="Supported platforms for the application"
    )

In [26]:
# Research_findings
class ResearchNote(BaseModel):
    query: str = Field(
        ...,
        description="Query for the websearch"
    )
    context: str = Field(
        ...,
        description="Compressed findings"
    )
    sources: List[str] = Field(
        ...,
        description="List of all resources"
    )

In [27]:
class AudienceResearchState(BaseModel):
    supervisor_messages: Annotated[Sequence[BaseMessage], add_messages] = Field(
        default_factory=list,
        description="History of messages from the supervisor"
    )
    tool_iteration: int = Field(
        0,
        description="Number of tool use"
    )
    campaign: Campaign = Field(
        ...,
        description="Campaign information"
    )
    target_audience: TargetAudience = Field(
        ...,
        description="Target audience information"
    )
    product: Product = Field(
        ...,
        description="Product information"
    )
    raw_research_notes: Annotated[Sequence[ToolMessage], operator.add] = Field(
        default_factory=list,
        description="List of conduct_research tool results"
    )
    research_notes: Annotated[List[ResearchNote], operator.add] = Field(
        default_factory=list,
        description="List of processed and cleaned conduct_research results"
    )

---
## Prompts

In [28]:
research_instructions_prompt = PromptTemplate(
    input_variables=["date", "campaign", "target_audience", "product"],
    template="""You are a psychological researcher. Your job is to coordinate and perform research for a marketing campaign. for context, today date is {date}.

<campaign_info>
campaign: {campaign}
target_audience: {target_audience}
product: {product}
</campaign_info>

<task>
Your primary task is to gather information trough research that can transform provided generic target audience demographics into actionable deep psychological profiles which enables precise, high-impact advertising campaigns.
You have a conduct_research tool that searches for answer of your queries in the web and brings the answers back to you. You are responsible for coordinating and performing the research using this tool so that at the end of your research you can answer the following questions:

- **Core Values: What fundamental beliefs shape this audience’s preferences and loyalties?**
- **Daily routines: What are the regular habits and daily schedules of this audience?**
- **Behavioral patterns: What are the typical online and offline behaviors and purchasing habits of this audience?**
- **Emotional triggers: what emotionally motivates the audience to pay attention, react, or take action?**
- **Decision-making process: how decisions are made, including factors like price sensitivity, reliance on peer reviews, or  influencers’ authority?**
- **Pain points/challenges: What unmet needs or problems does this audience face that are relevant to our campaign or product?**
</task>

<available_tools>
1. **conduct_research**: For finding the answer of your queries in the web
2. **think_tool**: For reflection and strategic planning during research

**CRITICAL: Use think_tool before calling conduct_research to plan your approach, and after each conduct_research to assess progress**
</available_tools>

<instructions>
Adopt the mindset of a professional researcher with limited time and resources. Follow these steps strictly to maximize research effectiveness and depth:

1. **Read all details of the campaign, audience, and product carefully before taking any action.**
2. **Before starting each research cycle:** Use think_tool to reflect and plan your next query. Define exactly what information you seek, based on the current research goal. If the research topic or question is too broad to answer directly, break it down into specific, focused sub-questions. Prioritize which sub-question to resolve first.
3. **Use conduct_research to search for answers about the targeted query/sub-question.**
4. **After each conduct_research step:** Use think_tool to process and analyze the results, assess whether the answer is complete or needs elaboration, and refine subsequent queries as needed. If the information found is still too generic, continue narrowing/clarifying your sub-question until actionable, detailed insight is obtained. If enough information has been collected to meaningfully address the current question, move on to the next question or subtopic.
</instructions>

<constraints>
**Tool Call Budgets:**
- Hard maximum: 10 conduct_research calls per research session.
- Limit: 3 conduct_research calls per main profiling question (Core Values, Daily routines, etc.).

**Stopping Rules:**
- If your last 2 searches return similar information, or if you can confidently answer the current question, stop further research for that question and move on.
- If you reach 10 conduct_research calls in total without sufficient results, stop and summarize findings.
</constraints>

<final_step>
At the end of your research, approve that you have gathered enough information for creating effective audience profile.
</final_step>
"""
)

compress_research_instructions_prompt = PromptTemplate(
    input_variables=["date", "query", "results"],
    template="""You are a research assistant that has conducted research on a topic by calling web searche tool. Your job is now to clean up the findings, but preserve all of the relevant statements and information that the researcher has gathered. For context, today's date is {date}.

<task>
Your main task is to process the raw tool call containing results from web search and produce a structured report that:
- Aggregates every relevant fact and statement from results, in a clean, well-organized format
- Assigns in-text numeric citations (e.g., [1], [2]) for each unique source or URL found in the tool results
- Concludes with a full "Sources" section—mapping all citation numbers to their corresponding URLs and titles, as found in the tool outputs
</task>

<guidelines>
1. Your report should restate all factual findings and information from the tool results verbatim — DO NOT paraphrase, summarize, or alter any relevant detail or data.
2. If the same fact or statement occurs in multiple sources, you may group them but cite all sources that claimed it (e.g., "...as reported in [1][2][3]").
3. Assign a unique sequential citation number to each URL/source that appears in the raw tool call results, in the order you first reference them.
5. Your output should be comprehensive. DO NOT omit or exclude any statement, number, name, or detail that could be relevant to the research question.
6. The final "Sources" section must list all citations, formatted: ["[1] Source Title: URL", [2] Source Title: URL, ...]
</guideline>

<research_result>
query: {query}
results: {results}
</research_result>
"""
)

---
## Tools

In [29]:
# Tool functions
def deduplicate_search_results(search_results: dict) -> dict:
    """
    Deduplicate search results by URL to avoid processing duplicate content.

    Args:
        search_results: List of search result dictionaries

    Returns:
        Dictionary mapping URLs to unique results
    """
    unique_results = {}

    for result in search_results["results"]:
        url = result['url']
        if url not in unique_results:
            unique_results[url] = result["content"]

    return unique_results

In [30]:
# Tool implementation
tavily_search = TavilySearch(tavily_api_key=settings.tavily_api_key.get_secret_value(), max_results=5, search_depth="advanced")

@tool
def conduct_research(query: str) -> dict:
    """
    Searches for information about a given query using Tavily search engine.

    Args:
        query(str): The query to search for.

    Returns:
        dict: A dictionary containing the search results and sources.
    """
    # Search for the query
    search_result = tavily_search.invoke(query)

    # Deduplicate the search results
    unique_results = deduplicate_search_results(search_result)

    return unique_results


@tool
def think_tool(reflection: str) -> str:
    """
    Tool for strategic reflection on research progress and decision-making.

    Use this tool after each search to analyze results and plan next steps systematically.
    This creates a deliberate pause in the research workflow for quality decision-making.

    When to use:
    - After receiving search results: What key information did I find?
    - Before deciding next steps: Do I have enough to answer comprehensively?
    - When assessing research gaps: What specific information am I still missing?
    - Before concluding research: Can I provide a complete answer now?

    Reflection should address:
    1. Analysis of current findings - What concrete information have I gathered?
    2. Gap assessment - What crucial information is still missing?
    3. Quality evaluation - Do I have sufficient evidence/examples for a good answer?
    4. Strategic decision - Should I continue searching or provide my answer?

    Args:
        reflection: Your detailed reflection on research progress, findings, gaps, and next steps

    Returns:
        Confirmation that reflection was recorded for decision-making
    """
    return f"Reflection recorded: {reflection}"

tools = [conduct_research, think_tool]
tools_by_name = {tool.name: tool for tool in tools}

---
## Nodes

In [31]:
# Create the model and bind tools
model = ChatOpenAI(api_key=settings.open_ai_api_key.get_secret_value(), temperature=0, model="gpt-4.1")
model_with_tools = model.bind_tools(tools)

In [32]:
def researcher_node(state: AudienceResearchState):
    """Agent node responsible for conducting research."""
    # Extract state variables
    campaign = state.campaign.model_dump_json()
    target_audience = state.target_audience.model_dump_json()
    product = {"name": state.product.name, "description": test_state.product.description}

    # Inject state variables into the researcher prompt
    prompt = research_instructions_prompt.format(
        date=get_date_string(),
        campaign=campaign,
        target_audience=target_audience,
        product=product
    )

    # Create messages history
    messages = [SystemMessage(content=prompt)] + state.supervisor_messages
    return {"supervisor_messages": model_with_tools.invoke(messages)}

In [33]:
def tool_node(state: AudienceResearchState):
    """
    Execute all tool calls from the previous LLM response.

    Executes all tool calls from the previous LLM responses.
    Returns the updated state with tool execution results.
    """
    tool_calls = state.supervisor_messages[-1].tool_calls

    # Execute all tool calls
    observations = []
    for tool_call in tool_calls:
        tool = tools_by_name[tool_call["name"]]
        observations.append(tool.invoke(tool_call["args"]))

    # Create tool message output
    tool_outputs = [
        ToolMessage(
            content=observation,
            name=tool_call["name"],
            tool_call_id=tool_call["id"],
            query=tool_call["args"]
        ) for observation, tool_call in zip(observations, tool_calls)
    ]

    tool_iter = state.tool_iteration + 1
    
    return {"supervisor_messages": tool_outputs, "tool_iteration": tool_iter}

In [34]:
# def compress_research_node(state: AudienceResearchState):
#     """
#     Compress the tool call result into structured output
#     """
#     last_research = state.raw_research_notes[-1]

#     # Add structure output to the model
#     model_with_structured_output = model.with_structured_output(ResearchNote)

#     # Create prompt
#     prompt = compress_research_instructions_prompt.format(
#         date=get_date_string(),
#         query=last_research.query,
#         results=last_research.content
#     )

#     compressed_research = model_with_structured_output.invoke(prompt)

#     # Create Tool message
#     tool_output = [ToolMessage(
#         content=compressed_research.context,
#         name=last_research.name,
#         tool_call_id=last_research.tool_call_id,
#         query=last_research.query,
#         sources=compressed_research.sources
#     )]

#     return{"research_notes": [compressed_research], "supervisor_messages": tool_output}

In [35]:
# Routing functions
def should_use_tool(state: AudienceResearchState) -> Literal["tool_node", "__end__"]:
    """
    Determine whether to continue research or provide the final answer.

    Determines whether the agent should continue the research loop or provide
    a final answer based on whether the LLM made tool calls.

    Returns:
        "tool_node": Continue to tool execution
        "__end__": Stop and craft the final asnwer
    """
    messages = state.supervisor_messages
    last_message = messages[-1]

    # if state.tool_iteration > 3:
    #     return "__end__"

    # If the LLM makes a tool call, continue to tool execution
    if last_message.tool_calls:
        return "tool_node"

    return "__end__"


# def should_compress_research(state: AudienceResearchState) -> Literal["compress_node", "researcher_node"]:
#     """
#     Determine whether to move to compress node to clean the research or move back to researcher.

#     Returns:
#         "compress_node": Continue to compress node
#         "researcher_node": Continue to researcher node
#     """
#     messages = state.supervisor_messages
#     last_message = messages[-1]

#     # If the LLM makes a tool call, continue to tool execution
#     if any(call["name"] == "think_tool" for call in getattr(last_message, "tool_calls", [])):
#         return "researcher_node"

#     return "compress_node"

In [36]:
builder = StateGraph(AudienceResearchState)

builder.add_node("researcher", researcher_node)
builder.add_node("tool_node", tool_node)
# builder.add_node("compress_node", compress_research_node)

builder.add_edge(START, "researcher")
builder.add_conditional_edges(
    "researcher",
    should_use_tool,
    {
        "tool_node": "tool_node",
        "__end__": END
    }
)
# builder.add_conditional_edges(
#     "tool_node", 
#     should_compress_research,
#     {
#         "compress_node": "compress_node",
#         "researcher_node": "researcher"
#     }
# )
builder.add_edge("tool_node", "researcher")
builder.add_edge("researcher", END)

graph = builder.compile()

In [37]:
# Test state
test_state = AudienceResearchState(
    supervisor_messages=[],
    campaign=Campaign(
        goal=CampaignGoal.AWARENESS,
        platform=AdPlatform.TIKTOK_REELS,
    ),
    target_audience=TargetAudience(
        gender=Gender.FEMALE,
        location=Location.USA,
        income_range=IncomeRange.LOWER_MIDDLE,
        age_range=AgeRange(
            min_age=25,
            max_age=35
        )
    ),
product=Product(
        name="Delisio - Your Personal Chef",
        description="Delisio is a personal chef and nutrition assistant. When a user signs up, they enter their age, weight, height, diet (e.g., vegan, keto), nutritional goal (e.g., weight loss, muscle gain), allergies, and equipment. Delisio then provides personalized recipes based on preferences and uses AI to personalize the user's food experience.",
        features={
            "Photo to Recipe": "User uploads a dish photo, specifies equipment and calorie preference, and Delisio generates a tailored recipe.",
            "Surprise Me": "User selects meal type, cuisine, and equipment, defines calorie goal; Delisio creates a personalized recipe (e.g., vegetarian Chinese breakfast).",
            "Nutrition Scanner": "User scans food; Delisio analyzes and reports nutrients, vitamins, and minerals."
        },
        supported_platforms=[ProductPlatform.ios, ProductPlatform.android]
    ))

In [38]:
state = graph.invoke(test_state, {"recursion_limit": 100})

In [39]:
state["tool_iteration"]

22

---
# Summarization

In [40]:
research_tool_messages = []
for message in state["supervisor_messages"]:
    if type(message) == ToolMessage:
        if message.name != 'think_tool':
            research_tool_messages.append({"query": message.query, "content": message.content})
        else:
            research_tool_messages.append({message.content})

In [41]:
# State
class SummaryState(BaseModel):
    messages: Sequence[BaseMessage] = Field(
        default_factory=list,
        description="Research Messages"
    )
    campaign: Campaign = Field(
        ...,
        description="Campaign information"
    )
    target_audience: TargetAudience = Field(
        ...,
        description="Target audience information"
    )
    product: Product = Field(
        ...,
        description="Product information"
    )

    report: Optional[str] = Field(
        None,
        description="Research report generates from research agent's tool calls"
    )

In [42]:
summary_instructions_prompt = PromptTemplate(
    input_variables=["date", "campaign", "target_audience", "product", "research"],
    template="""You are an expert technical writer specialized in psychological research reports for marketing applications. You will receive a complete process of a psychological research focused on profiling a specific target audience. For context, today’s date is {date}.

<campaign_information>
- **Campaign:** {campaign}
- **Target Audience:** {target_audience}
- **Product:** {product}
</campaign_information>

<research_methodology>
The research was conducted in a step-by-step loop where the researcher used a think tool to reflect, analyze findings, and plan each next move before and after every web search; this approach allowed for continuous adjustment and deeper investigation, ensuring each search was purposeful and insights were fully analyzed before moving forward.
</research_methodology>

<objective>
Produce a comprehensive, actionable, and well-organized report for the marketing team that distills all relevant insights uncovered during the research. The report should enable marketing professionals to design precise, psychologically informed campaigns targeting the identified audience.
</objective>

<document_structure>
- Begin with a **Title** and a **Table** including campaign(goal, platform as separate columns), target audience demographics (gender, location, etc.. as separate column), and product (use a Markdown table for clarity).
- Follow with an **Executive Summary** that briefly introduces the key findings and sets expectations for the rest of the report.
- Organize the main sections as follows (use these exact headings in order):
    - Core Values
    - Daily Routines
    - Behavioral Patterns
    - Emotional Triggers
    - Decision-making Process
    - Pain Points & Challenges
- If the research reveals other themes relevant to audience psychology, add extra sections as needed (with meaningful, specific headings).
- Conclude with a **Conclusion** section that synthesizes the most actionable insights relevant for campaign planning.
</document_structure>

<citation_rules>
- Use in-text, numbered citations in [#] format immediately after the corresponding fact or statement.
- Number sources sequentially (1, 2, 3...) — never skip or reuse citation numbers.
- End with a **Sources** section (### Sources) listing each referenced source as: [#] Source Title: URL  
(one per line, in markdown, no blank lines between).
</citation_rules>

<guidelines>
- Provide concrete, actionable insights—avoid generic or surface-level statements.
- Reference specific facts, statistics, and examples from the research.
- Where helpful, use Markdown tables or bullet lists within sections for clarity.
- Focus on creating value for marketing practitioners: emphasize implications for messaging, positioning, and targeting wherever possible.
- Avoid all self-referential language (“As the writer…” or “This report will…”).
- Write in a clear, formal, and objective tone.
- Do not include any statements describing report authorship, process, or methodology in the main report body.
</guidelines>

<research>
here is the whole process of the research:
{research}
</research>
"""
)

In [43]:
# Node
def summarize_report(state: SummaryState):
    campaign = state.campaign.model_dump_json()
    target_audience = state.target_audience.model_dump_json()
    product = {"name": state.product.name, "description": test_state.product.description}
    messages = state.messages

    # Extract tool messages
    research_tool_messages = []
    for message in state.messages:
        if type(message) == ToolMessage:
            if message.name != 'think_tool':
                research_tool_messages.append({"query": message.query, "content": message.content})
            else:
                research_tool_messages.append({message.content})

    # create promtp
    prompt = summary_instructions_prompt.format(
        date=get_date_string(),
        campaign=campaign,
        target_audience=target_audience,
        product=product,
        research=research_tool_messages
    )

    response = model.invoke([HumanMessage(content=prompt)])

    return{"report": response}

In [44]:
# graph
summary_builder = StateGraph(SummaryState)

summary_builder.add_node("summarize_report", summarize_report)

summary_builder.add_edge(START, "summarize_report")
summary_builder.add_edge("summarize_report", END)

summary_graph = summary_builder.compile()

In [45]:
summary_state = SummaryState(
    messages=state["supervisor_messages"],
    campaign=state["campaign"],
    target_audience=state["target_audience"],
    product=state["product"]
)

In [46]:
summary_state = summary_graph.invoke(summary_state)

In [47]:
from IPython.display import Markdown, display
display(Markdown(summary_state["report"].content))


# Psychological Profile Report: US Women Aged 25–35 for Delisio TikTok Awareness Campaign

| Campaign Goal | Platform      | Gender | Location       | Income Range | Age Range | Product Name                | Product Description                                                                                                                        |
|:--------------|:-------------|:-------|:---------------|:-------------|:----------|:----------------------------|:-------------------------------------------------------------------------------------------------------------------------------------------|
| Awareness     | TikTok Reels | Female | United States  | $30k–$60k    | 25–35     | Delisio – Your Personal Chef | AI-powered personal chef and nutrition assistant providing personalized recipes based on user data, preferences, and goals.                |

---

## Executive Summary

Women aged 25–35 in the US with moderate incomes are digitally savvy, value-driven, and time-constrained. They seek convenience, personalization, and trustworthy guidance in their food and nutrition choices. Social media—especially TikTok—plays a central role in shaping their behaviors, emotional responses, and purchase decisions. This report distills actionable psychological insights to inform precise, resonant campaign strategies for Delisio, focusing on core values, daily routines, behavioral patterns, emotional triggers, decision-making processes, and pain points.

---

## Core Values

- **Health & Wellness:** Weight management and stress reduction are top health priorities. There is a strong emphasis on balanced nutrition, with attention to protein, fiber, and limiting added sugar. Mental health and self-care are increasingly valued alongside physical health [1][2].
- **Convenience & Efficiency:** Solutions that save time and reduce effort are highly prized. This group seeks out products and services that fit seamlessly into busy lives [3][4].
- **Personalization:** Customization is expected—whether in food, digital experiences, or wellness solutions. Brands that offer tailored recommendations and experiences earn greater loyalty [5][6].
- **Technology Adoption:** High comfort with digital tools and apps for meal planning, shopping, and health tracking. User-friendly, intuitive technology is a baseline expectation [7].
- **Sustainability & Transparency:** Preference for brands that are transparent about sourcing, ingredients, and business practices. Social responsibility and sustainability are important differentiators [3][8].
- **Authenticity & Trust:** Loyalty is built on authentic, relatable messaging and influencer partnerships. Peer and influencer recommendations are trusted sources of information [9][10].

**Implications:** Messaging should highlight Delisio’s convenience, personalization, and alignment with health, wellness, and sustainability values. Authentic storytelling and influencer partnerships are critical.

---

## Daily Routines

- **Work & Time Constraints:** Most women in this group work full-time (often 40+ hours/week), with structured mornings and evenings. Meal preparation is typically squeezed into evenings or weekends [11][12].
- **Meal Prep Habits:** Average time spent cooking is 40–45 minutes per day, with a preference for efficient, planned meals. Many use digital tools (apps, recipe folders) to streamline meal planning and prep [13][14].
- **Family & Multitasking:** For those with children, routines include daycare drop-offs, family meals, and multitasking between work, childcare, and self-care [11].
- **Weekend Planning:** Some prep lunches or meals in advance on weekends to save time during the week [13].
- **Device Usage:** High daily engagement with smartphones and apps for recipe discovery, meal planning, and grocery shopping [7][13].

**Implications:** Position Delisio as a time-saving, stress-reducing solution that fits into busy routines. Highlight features like meal planning, quick recipes, and digital convenience.

---

## Behavioral Patterns

- **Social Media Engagement:** TikTok is a primary platform for food, nutrition, and wellness content. 42% of Millennials shop directly on TikTok; social commerce is mainstream for this group [15].
- **Influencer & Peer Impact:** Influencer content, peer reviews, and trending challenges drive discovery and adoption of new products [9][16].
- **Online & Offline Shopping:** While online shopping is growing, a majority still buy groceries in-person at least once a week. Nutrition labels are checked both online and in-store [17][18].
- **Content Preferences:** Engaging, visually appealing, and authentic content (e.g., recipe videos, transformation stories) performs best. Trending formats and challenges increase engagement [19].
- **Adoption Journey:** Discovery often starts on social media, with purchases completed in-app or via direct links. Convenience and seamless digital experiences are expected [15][20].

**Implications:** Leverage TikTok’s native formats, trending challenges, and influencer partnerships. Ensure a seamless path from discovery to action, with clear calls to action and easy onboarding.

---

## Emotional Triggers

- **Social Validation & Trends:** Engagement is driven by trending content, influencer recommendations, and social proof. Participation in challenges and viral trends provides a sense of belonging [21][22].
- **Body Image & Self-Improvement:** Content promising weight loss, self-care, or improved appearance is highly engaging, but can also trigger guilt or negative emotions if not handled sensitively [23][24].
- **Stress Relief & Comfort:** Food content can trigger cravings and impulsive eating, especially when users seek emotional or social fulfillment [25].
- **Empowerment & Positivity:** Messages that promote self-acceptance, confidence, and practical wellness tips resonate positively and foster loyalty [26].
- **Information Overload:** Conflicting advice and overwhelming information can cause anxiety and decision fatigue [27].

**Implications:** Use empowering, positive messaging that emphasizes achievable wellness, self-care, and personalization. Avoid guilt-based or restrictive language. Highlight community and social validation aspects.

---

## Decision-Making Process

- **Influencer Authority:** Authenticity, trust, and perceived expertise of influencers are critical. 72% of respondents agree that influencer authenticity is important in purchase decisions [28][29].
- **Peer Reviews & Social Proof:** Recommendations from friends, family, and online reviews significantly impact choices [30].
- **Price Sensitivity & Value:** While price matters, alignment with personal values (health, sustainability) and trust in the source often outweigh cost alone [31].
- **Convenience:** Ease of use, quick onboarding, and seamless integration into daily life are decisive factors [32].
- **Personalized Recommendations:** Tailored suggestions and experiences increase purchase likelihood [6][33].

**Implications:** Partner with credible influencers, showcase authentic testimonials, and provide clear, personalized value propositions. Streamline the user journey from discovery to sign-up.

---

## Pain Points & Challenges

- **Time Scarcity:** Busy schedules and long work hours make it difficult to plan, shop for, and prepare healthy meals [34][35].
- **Information Overload:** Conflicting nutrition advice, especially from social media, leads to confusion and decision fatigue [27][36].
- **Consistency Struggles:** Maintaining healthy habits is challenging due to time pressure, stress, and the lure of convenience foods [35].
- **Personalization Gaps:** Many solutions are too generic; there is a strong desire for tailored guidance that fits individual needs and goals [6][37].
- **Emotional Barriers:** Guilt, feelings of deprivation, and body image concerns can undermine motivation and adherence [23][38].

**Implications:** Emphasize Delisio’s ability to save time, cut through information noise, and deliver truly personalized, supportive guidance. Use messaging that normalizes setbacks and encourages progress over perfection.

---

## Conclusion

Women aged 25–35 in the US with moderate incomes are health-conscious, digitally engaged, and value-driven, but face significant time and information barriers to healthy eating. They respond to authentic, empowering content and trust influencers and peers for guidance. Campaigns for Delisio should focus on convenience, personalization, and positive, community-driven messaging—delivered through TikTok’s native formats and influencer partnerships. Addressing pain points around time, information overload, and emotional barriers will position Delisio as an indispensable, trusted ally in their wellness journey.

---

### Sources

[1] Top Trends in Women’s Health and Nutrition: https://www.glanbianutritionals.com/en/nutri-knowledge-center/insights/top-trends-womens-health-and-nutrition  
[2] Women’s Health Care Utilization and Costs: https://www.kff.org/womens-health-policy/womens-health-care-utilization-and-costs-findings-from-the-2020-kff-womens-health-survey/  
[3] Changing Food Habits of Millennials: https://www.restroworks.com/blog/changing-food-habits-of-millennials-are-impacting-the-restaurant-business/  
[4] The Shelf: Millennial Women: https://www.theshelf.com/the-blog/millennial-women/  
[5] Millennial Food Preferences: https://gfs.com/en-us/ideas/millennial-food-preferences/  
[6] TikTok Consumer Purchasing Behavior Report 2024-2025: https://www.britopian.com/wp-content/uploads/2025/03/TikTok-Consumer-Purchasing-Behavior-Report-2024-2025.pdf  
[7] Healthy Eating Guide for Women: https://www.hopkinsmedicine.org/-/media/files/health/ebooks/healthy-eating-guide-women.ashx  
[8] Sustainable Consumer Behaviors by Generation: https://pmc.ncbi.nlm.nih.gov/articles/PMC10888481/  
[9] Social Media Influencers’ Impact on Consumer Purchasing Decisions: https://ijsrem.com/download/the-impact-of-social-media-influencers-on-consumer-buying-decisions-related-to-food-products/  
[10] Social Media Influencers’ Impact on Consumer Purchasing Decisions (Conference Paper): https://docs.rwu.edu/cgi/viewcontent.cgi?article=1157&context=nyscaproceedings  
[11] Working Women’s Daily Routines (Quora): https://www.quora.com/As-a-working-woman-what-is-your-daily-routine-schedule-and-how-do-you-manage-and-utilize-time-to-be-multitasking  
[12] Project EAT-III Survey: https://pmc.ncbi.nlm.nih.gov/articles/PMC3464955/  
[13] Nutrition Journal: https://nutritionj.biomedcentral.com/articles/10.1186/s12937-018-0347-9  
[14] Cooking Trends 2023 ATUS: https://cdn.nutrition.org/article/S2475-2991(25)02991-9/pdf  
[15] IFIC 2023 Food & Health Report: https://ific.org/wp-content/uploads/2023/05/IFIC-2023-Food-Health-Report.pdf  
[16] Influence of TikTok Trends on Buying Intentions: https://rsisinternational.org/journals/ijriss/articles/influence-of-tiktok-trends-on-the-buying-intentions-of-gen-zs/  
[17] Online Grocery Shopping Patterns: https://pmc.ncbi.nlm.nih.gov/articles/PMC9609768/  
[18] Food Purchasing and Demographics: https://pmc.ncbi.nlm.nih.gov/articles/PMC4331127/  
[19] Food Content Engagement on TikTok: https://libstore.ugent.be/fulltxt/RUG01/003/218/207/RUG01-003218207_2024_0001_AC.pdf  
[20] Retail Dive: Social Media Influence on Shopping: https://www.retaildive.com/news/generation-z-social-media-influence-shopping-behavior-purchases-tiktok-instagram/652576/  
[21] Social Media Significantly Influences Nutritional Choices: https://www.uri.edu/news/2025/04/social-media-significantly-influences-nutritional-choices-emotional-well-being-students-study-shows/  
[22] Uses and Gratifications Theory in Food Content: https://libstore.ugent.be/fulltxt/RUG01/003/218/207/RUG01-003218207_2024_0001_AC.pdf  
[23] TikTok Nutrition Content and Body Image: https://www.cnbc.com/2022/11/09/tiktok-nutrition-related-content-may-contribute-to-disordered-eating.html  
[24] Negative Portrayal of Body Image on Social Media: https://pmc.ncbi.nlm.nih.gov/articles/PMC11504184/  
[25] Emotional Eating and Social Media: https://pmc.ncbi.nlm.nih.gov/articles/PMC9701005/  
[26] Positive Body Image Messaging: https://pmc.ncbi.nlm.nih.gov/articles/PMC11504184/  
[27] Nutrition Information Overload: https://pmc.ncbi.nlm.nih.gov/articles/PMC10346027/  
[28] Social Media Influencers’ Impact on Consumer Buying Decisions: https://ijsrem.com/download/the-impact-of-social-media-influencers-on-consumer-buying-decisions-related-to-food-products/  
[29] Social Media Influencers’ Impact on Consumer Purchasing Decisions (Conference Paper): https://docs.rwu.edu/cgi/viewcontent.cgi?article=1157&context=nyscaproceedings  
[30] Retail Dive: Social Media Influence on Shopping: https://www.retaildive.com/news/generation-z-social-media-influence-shopping-behavior-purchases-tiktok-instagram/652576/  
[31] Sustainable Consumer Attitudes: https://www.mdpi.com/2071-1050/16/13/5471  
[32] IFIC 2023 Food & Health Report: https://ific.org/wp-content/uploads/2023/05/IFIC-2023-Food-Health-Report.pdf  
[33] Influence of TikTok Trends on Buying Intentions: https://rsisinternational.org/journals/ijriss/articles/influence-of-tiktok-trends-on-the-buying-intentions-of-gen-zs/  
[34] Healthy Eating and Women: https://womenshealth.gov/healthy-eating/healthy-eating-and-women  
[35] Project EAT-III Survey: https://pmc.ncbi.nlm.nih.gov/articles/PMC3464955/  
[36] Nutrition Information Overload: https://pmc.ncbi.nlm.nih.gov/articles/PMC10346027/  
[37] Top Trends in Women’s Health and Nutrition: https://www.glanbianutritionals.com/en/nutri-knowledge-center/insights/top-trends-womens-health-and-nutrition  
[38] MD Anderson: 5 Barriers to Diet Change: https://www.mdanderson.org/cancerwise/5-barriers-to-diet-change-and-how-to-overcome-them.h00-159778023.html  


In [48]:
import os
from markdown_pdf import MarkdownPdf, Section

# Create PDF instance
pdf = MarkdownPdf(toc_level=2)

# Add your markdown content
pdf.add_section(Section(summary_state["report"].content))

# Set metadata (optional)
pdf.meta["title"] = "Delisio USA Psychological Profile Report"
pdf.meta["author"] = "Reseacher Agent"

# Save to home directory (writable location)
output_path = os.path.expanduser("~/Documents/Delisio USA Psychological Profile Report.pdf")
pdf.save(output_path)

print(f"PDF saved successfully to: {output_path}")


2025-10-17 14:09:44,965 - markdown_it.rules_block.code - DEBUG - entering code: StateBlock(line=0,level=0,tokens=0), 0, 133, False
2025-10-17 14:09:44,965 - markdown_it.rules_block.fence - DEBUG - entering fence: StateBlock(line=0,level=0,tokens=0), 0, 133, False
2025-10-17 14:09:44,966 - markdown_it.rules_block.blockquote - DEBUG - entering blockquote: StateBlock(line=0,level=0,tokens=0), 0, 133, False
2025-10-17 14:09:44,966 - markdown_it.rules_block.hr - DEBUG - entering hr: StateBlock(line=0,level=0,tokens=0), 0, 133, False
2025-10-17 14:09:44,967 - markdown_it.rules_block.list - DEBUG - entering list: StateBlock(line=0,level=0,tokens=0), 0, 133, False
2025-10-17 14:09:44,967 - markdown_it.rules_block.reference - DEBUG - entering reference: StateBlock(line=0,level=0,tokens=0), 0, 133, False
2025-10-17 14:09:44,967 - markdown_it.rules_block.html_block - DEBUG - entering html_block: StateBlock(line=0,level=0,tokens=0), 0, 133, False
2025-10-17 14:09:44,967 - markdown_it.rules_block.h

PDF saved successfully to: /Users/peymankhodabandehlouei/Documents/Delisio USA Psychological Profile Report.pdf
